In [ ]:
:dep plotters = { version = "0.3.7", default-features = false, features = ["evcxr", "all_series", "all_elements"] }
:dep itertools = "0.13.0"
:dep simpledie = { path = ".." }

In [ ]:
extern crate plotters;

use std::cmp::Ord;
use std::ops::Add;
use std::ops::Mul;

use itertools::Itertools;
use plotters::prelude::*;
use plotters::evcxr::SVGWrapper
use simpledie::Die;
use simpledie::{D100, D20, D12, D8, D6, D4, D2};

In [ ]:
fn plot_die<K>(die: &Die<K>) -> SVGWrapper
where
    K: Copy,
    u64: From<K>,
{
    let data: Vec<(u64, f64)> = die.probabilities().unwrap();
    let x0 = data.first().unwrap().0;
    let x1 = data.last().unwrap().0;
    let m = data.iter().map(|(_, c)| *c).reduce(f64::max).unwrap();

    evcxr_figure((640, 480), |root| {
        root.fill(&WHITE)?;
        let mut chart = ChartBuilder::on(&root)
            .x_label_area_size(35)
            .y_label_area_size(40)
            .margin(5)
            .build_cartesian_2d((x0..x1).into_segmented(), 0f64..m)?;
        chart
            .configure_mesh()
            .disable_x_mesh()
            .draw()?;
        chart.draw_series(
            Histogram::vertical(&chart)
                .style(BLUE.filled())
                .margin(1)
                .data(data.iter().map(|&(x, y)| (x, y))),
        )?;    
        Ok(())
    })
}

In [ ]:
let save_dc = 8 + 3 + 5;
let ability_mod = 4;
let hit = D6.repeat(8, Add::add);
let d = D20
    .combine_with(&hit, |x, y| if x + ability_mod < save_dc { y } else { y / 2 })
;
plot_die(&d)